In [ ]:
from pdb import set_trace

def multi(dispatch_fn):
    def _inner(*args, **kwargs):
        return _inner.__multi__.get(
            dispatch_fn(*args, **kwargs),
            _inner.__multi_default__
        )(*args, **kwargs)

    _inner.__multi__ = {}
    _inner.__multi_default__ = lambda *args, **kwargs: None  # Default default
    return _inner

def method(dispatch_fn, dispatch_key=None):
    def apply_decorator(fn):
        if dispatch_key is None:
            # Default case
            dispatch_fn.__multi_default__ = fn
        else:
            dispatch_fn.__multi__[dispatch_key] = fn
        return dispatch_fn
    return apply_decorator

In [ ]:
from dataclasses import dataclass
from enum import Enum
from functools import wraps

TfmType = Enum('TfmType','Coord Pixel')

@dataclass
class Item(object):
    tfm_type:TfmType
    foo: str

@dataclass
class BoxItem(Item):
    box:str

In [ ]:
def whatever(func):
    @wraps(func)
    def apply_to_list(xs, *args, **kwargs):
        return [func(x, *args, **kwargs) for x in xs]
    return apply_to_list

@whatever
@multi
def zoom(x, *args, **kwargs):
    #print('dispatch', x.tfm_type, x.__class__, x.foo)
    return x.tfm_type, x.__class__, x.foo

@method(zoom,(TfmType.Coord, Item, 'foo'))
def zooma(x, *args, **kwargs):
    print('coord item and foo', args, kwargs)
    
@method(zoom,(TfmType.Coord, Item, 'bar'))
def zoomb(x, *args, **kwargs):
    print('coord item and bar', args, kwargs)

@method(zoom,(TfmType.Coord, BoxItem, 'foo'))
def zoomc(x, *args, **kwargs):
    print('coord BoxItem and foo', args, kwargs)
    
@method(zoom,(TfmType.Coord, BoxItem, 'bar'))
def zoomd(x, *args, **kwargs):
    print('coord BoxItem and bar', args, kwargs)

@method(zoom,(TfmType.Pixel, Item, 'bar'))
def zoome(x, *args, **kwargs):
    print('pix, item or box and bar', args, kwargs)

@method(zoom)  # Default
def zoomf(*args, **kwargs):
    return "huh?"


In [ ]:
a = Item(TfmType.Coord, 'foo')
b = Item(TfmType.Pixel, 'bar')
c = Item(TfmType.Pixel, 'foo')
d = BoxItem(TfmType.Coord, 'foo', 'a box')
e = BoxItem(TfmType.Pixel, 'bar', 'yep a box')
f = BoxItem(TfmType.Pixel, 'foo', 'foo bar')

In [ ]:
zoom([a,b,c,d,e,f], dispatch_is='cool')